Code based on the actuarial data science tutorial and paper:
https://github.com/actuarial-data-science/Tutorials/tree/master/6%20-%20Lee%20and%20Carter%20go%20Machine%20Learning%20Recurrent%20Neural%20Networks

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Data

 The data is divided by genders, and
 the raw mortality rates are denoted by mx, see line 7 of Listing 1. Since we are going to model
 log-mortality rates the raw morality rates need to be strictly positive. In the original data from
 the HMD there were 10 observations with mx = 0, these observations have been replaced
 by a mortality rate average over all available countries in the HMD (of the same age x, in the
 same calendar year t and of the same gender). These modi ed raw mortality rates received a
 flag, indicated on line 6 of Listing 1

In [7]:
mort_data_raw = pd.read_csv("CHE_mort.csv", sep=";")
mort_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13400 entries, 0 to 13399
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Gender        13400 non-null  object 
 1   Year          13400 non-null  int64  
 2   Age           13400 non-null  int64  
 3   Country       13400 non-null  object 
 4   imputed_flag  13400 non-null  bool   
 5   mx            13400 non-null  float64
 6   logmx         13400 non-null  float64
dtypes: bool(1), float64(2), int64(2), object(2)
memory usage: 641.3+ KB


imputed_flag
False    13390
True        10
Name: count, dtype: int64